In [7]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceHubEmbeddings,HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.utilities import ApifyWrapper
from langchain.document_loaders import TextLoader
from langchain.chains import VectorDBQA,RetrievalQA,LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AI21
from langchain.prompts import PromptTemplate
import import_ipynb
from Config import *
import warnings
warnings.filterwarnings("ignore")

In [8]:
loader = TextLoader("sample.txt",encoding='utf-8')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=900, chunk_overlap=2, separator='\n')
docs = text_splitter.split_documents(documents)


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings= HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)#, persist_directory=persist_directory)


No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [9]:
llm = AI21(temperature=0, ai21_api_key= key_value_dict['ai21_Key'])

# set prompt template
prompt_template = """Use the following pieces of context to answer the question at the end give me the confidence score for generated answer. If you don't know the answer, just say that you don't know, 
don't try to make up an answer.
{context}
Question: {question}
Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [14]:
query = "Help me in summarizing the text in a bullet point format"
dcs = vectordb.similarity_search(query,k=3)

print(dcs,sep = "\n")


[Document(page_content="You'll need to integrate your chatbot with ServiceNow's RESTful API. To do this:\nCreate a ServiceNow developer account if you don't have one.\nObtain the necessary API credentials (API key or OAuth tokens) from ServiceNow.\nImplement code in your Azure Web App to make HTTP requests to ServiceNow's API to retrieve or update records, create incidents, etc. This code should be part of your chatbot's logic.\nBot Logic:\nYour chatbot logic should now include actions that interact with ServiceNow. For example, when a user asks the chatbot to create a ServiceNow incident, the bot should make an API call to create the incident in ServiceNow and respond to the user with the incident number or status.\nTesting:\nThoroughly test your chatbot's integration with ServiceNow. Ensure that it can handle different user requests and interact effectively with ServiceNow's API.\nSecurity and Authentication:", metadata={'source': 'sample.txt'}), Document(page_content="Ensure that yo

In [15]:
from langchain.chains.question_answering import load_qa_chain

In [16]:
chain = load_qa_chain(llm=llm)
response =  chain.run(input_documents = docs,question = query)
print(response,sep='\n')


* Create an Azure Web App
* Develop or Configure Your Chatbot
* Set up Communication Channels
* ServiceNow API Integration
  * Create a ServiceNow developer account
  * Obtain the necessary API credentials (API key or OAuth tokens) from ServiceNow
  * Implement code in your Azure Web App to make HTTP requests to ServiceNow's API to retrieve or update records, create incidents, etc.
* Bot Logic
  * Your chatbot logic should now include actions that interact with ServiceNow
  * For example, when a user asks the chatbot to create a ServiceNow incident, the bot should make an API call to create the incident in ServiceNow and respond to the user with the incident number or status.
* Testing
  * Thoroughly test your chatbot's integration with ServiceNow
  * Ensure that it can handle different user requests and interact effectively with ServiceNow's API
* Security and Authentication
  * Ensure that your Azure Web App and the communication with ServiceNow are secure
  * This may involve using